In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# miniFraudFinder - Feature Engineering (batch)


## Overview

[miniFraudFinder](https://github.com/jrdetorre-google/minifraudfinder) is a series of labs on how to build a  fraud detection system on Google Cloud. Throughout the miniFraudFinder labs, you will learn how to read historical bank transaction data stored in data warehouse,  perform exploratory data analysis (EDA), do feature engineering,  train a model u, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model and monitor your model.

### Objective

As you engineer features for model training, it's important to consider how the features are computed when making predictions with new data. For online predictions, you may have features that can be pre-computed via _batch feature engineering_. For these miniFraudfinder labs, for computing features based on the last n _days_, you will use _batch_ feature engineering in BigQuery.

This notebook shows how to generate new features on bank transactions by customer and terminal over the last n days, by doing batch feature engineering in SQL with BigQuery. 

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)


Steps performed in this notebook:

- Build customer and terminal-related features in BigQuery


### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [1]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)


BUCKET_NAME          = "orange-ml-demo-fraudfinder"
PROJECT              = "orange-ml-demo"
REGION               = "us-central1"
ID                   = "zv7r3"
FEATURESTORE_ID      = "fraudfinder_zv7r3"
MODEL_NAME           = "ff_model"
ENDPOINT_NAME        = "ff_model_endpoint"
TRAINING_DS_SIZE     = "1000"



### Import libraries

In [2]:
# General
import datetime as dt
import json
import os
import random
import sys
import time
from datetime import datetime, timedelta
from typing import List, Union

# Data Engineering
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

# Vertex AI and Vertex AI Feature Store
from google.cloud import aiplatform as vertex_ai
from google.cloud import bigquery

### Define constants

In [46]:
# Define the date range of transactions for feature engineering (last 10 days up until yesterday)
YESTERDAY = datetime.today() - timedelta(days=30)
YEAR_MONTH_PREFIX = YESTERDAY.strftime("%Y-%m")
DATAPROCESSING_START_DATE = (YESTERDAY - timedelta(days=10)).strftime("%Y-%m-%d")
DATAPROCESSING_END_DATE = YESTERDAY.strftime("%Y-%m-%d")

# Define BiqQuery dataset and tables to calculate features.
RAW_BQ_TRANSACTION_TABLE_URI = f"{PROJECT_ID}.tx.tx"
RAW_BQ_LABELS_TABLE_URI = f"{PROJECT_ID}.tx.txlabels"
FEATURES_BQ_TABLE_URI = f"{PROJECT_ID}.tx.wide_features_table"

# Define Vertex AI Feature store settings.
CUSTOMERS_TABLE_NAME = f"customers_{DATAPROCESSING_END_DATE.replace('-', '')}"
CUSTOMERS_BQ_TABLE_URI = f"{PROJECT_ID}.tx.{CUSTOMERS_TABLE_NAME}"
TERMINALS_TABLE_NAME = f"terminals_{DATAPROCESSING_END_DATE.replace('-', '')}"                    
TERMINALS_BQ_TABLE_URI = f"{PROJECT_ID}.tx.{TERMINALS_TABLE_NAME}"
ONLINE_STORAGE_NODES = 1
FEATURE_TIME = "feature_ts"
CUSTOMER_ENTITY_ID = "customer_entity"
TERMINAL_ENTITY_ID = "terminal_entity"

### Helpers

Define a set of helper functions to run BigQuery query and create features. 

In [4]:
def run_bq_query(sql: str, show=False) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
        show: A flag to show query result in a Pandas Dataframe
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    result = client_result.result()
    print(f"Finished job_id: {job_id}")
    
    if show:
        df = result.to_arrow().to_pandas()
        return df

## Feature Engineering

### Define customer and terminal-related features for batch feature engineering

In this section, you will create features, based on historical customer behaviour and historical terminal activities. This features will be batch-generated using SQL in BigQuery, where the historical data is stored.

The query below will calculate 2 sets of features: 

1. **Customer-related features**: which describes the spending behaviour of customer within 1, 7 and 15 days time windows using number of transactions and average amount spent in dollars ($)

2. **Terminal-related features** which describes the risk of a given terminal to be exposed to fraudulent transactions within 1, 7 and 15 days using average number of fraudulent transactions in dollars ($), the number of transactions and risk index. One thing to note is that you will add some delay to take into account time that would pass between the time of transaction and the result of fraud investigation or customer claim.

You will use one month of transaction data starting from the end of January and going back to compute the features.

Below is the schema you should expect to see, after doing the batch feature engineering in BigQuery:

|feature_time           |customer_id| customer batch features   |
|-----------------------|-----------|---------------------------|
|2022-01-01 17:20:15 UTC|1          |(e.g., nb_tx,  avg_tx)     |
|2022-01-02 12:08:40 UTC|2          |(e.g., nb_tx,  avg_tx)     |
|2022-01-03 17:30:48 UTC|3          |(e.g., nb_tx,  avg_tx)     |


|feature_time           |terminal_id| terminal batch features|
|-----------------------|-----------|------------------------|
|2022-01-01 17:20:15 UTC|12345      |(e.g., risk_x_days)     |
|2022-01-02 12:08:40 UTC|26789      |(e.g., risk_x_days)     |
|2022-01-03 17:30:48 UTC|101112     |(e.g., risk_x_days)     |


#### Create the query to create batch features

Date settings to be used:

In [5]:
print(f"""
DATAPROCESSING_START_DATE: {DATAPROCESSING_START_DATE}
DATAPROCESSING_END_DATE: {DATAPROCESSING_END_DATE}
""")


DATAPROCESSING_START_DATE: 2023-12-10
DATAPROCESSING_END_DATE: 2023-12-20



##### Customer feature table

Customer table SQL query string:

In [9]:
create_customer_batch_features_query = f"""
CREATE OR REPLACE TABLE `{CUSTOMERS_BQ_TABLE_URI}` AS
WITH
  -- query to join labels with features -------------------------------------------------------------------------------------------
  get_raw_table AS (
  SELECT
    raw_tx.TX_TS,
    raw_tx.TX_ID,
    raw_tx.CUSTOMER_ID,
    raw_tx.TERMINAL_ID,
    raw_tx.TX_AMOUNT,
    raw_lb.TX_FRAUD
  FROM (
    SELECT
      *
    FROM
      `{RAW_BQ_TRANSACTION_TABLE_URI}`
    WHERE
      DATE(TX_TS) BETWEEN DATE_SUB("{DATAPROCESSING_END_DATE}", INTERVAL 15 DAY) AND "{DATAPROCESSING_END_DATE}"
    ) raw_tx
  LEFT JOIN 
    `{RAW_BQ_LABELS_TABLE_URI}` as raw_lb
  ON raw_tx.TX_ID = raw_lb.TX_ID),

  -- query to calculate CUSTOMER spending behaviour --------------------------------------------------------------------------------
  get_customer_spending_behaviour AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    TX_AMOUNT,
    TX_FRAUD,
    
    # calc the number of customer tx over daily windows per customer (1, 7 and 15 days, expressed in seconds)
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 86400 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_1DAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_7DAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_14DAY_WINDOW,
      
    # calc the customer average tx amount over daily windows per customer (1, 7 and 15 days, expressed in seconds, in dollars ($))
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 86400 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_14DAY_WINDOW,
  FROM get_raw_table)

# Create the table with CUSTOMER and TERMINAL features ----------------------------------------------------------------------------
SELECT
  PARSE_TIMESTAMP("%Y-%m-%d %H:%M:%S", FORMAT_TIMESTAMP("%Y-%m-%d %H:%M:%S", TX_TS, "UTC")) AS feature_ts,
  CUSTOMER_ID AS customer_id,
  CAST(CUSTOMER_ID_NB_TX_1DAY_WINDOW AS INT64) AS customer_id_nb_tx_1day_window,
  CAST(CUSTOMER_ID_NB_TX_7DAY_WINDOW AS INT64) AS customer_id_nb_tx_7day_window,
  CAST(CUSTOMER_ID_NB_TX_14DAY_WINDOW AS INT64) AS customer_id_nb_tx_14day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_1day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_7day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_14DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_14day_window,
FROM
  get_customer_spending_behaviour
"""

##### Run the query 

You create the customer features table

In [10]:
run_bq_query(create_customer_batch_features_query)

Finished job_id: 910584f0-1547-42a8-b44c-e806e20d0367


##### Inspect the result 

You can query some data rows to validate the result of the query

In [11]:
run_bq_query(
    f"SELECT * FROM `{CUSTOMERS_BQ_TABLE_URI}` LIMIT 10",
    show=True
)

Finished job_id: 6b152164-4aa7-4913-a608-37c20db0cdbb


,feature_ts,customer_id,customer_id_nb_tx_1day_window,customer_id_nb_tx_7day_window,customer_id_nb_tx_14day_window,customer_id_avg_amount_1day_window,customer_id_avg_amount_7day_window,customer_id_avg_amount_14day_window
0,2023-12-20 19:26:28+00:00,0065747991442579,2,14,37,12.530000,15.751429,16.534054
1,2023-12-20 14:12:29+00:00,0346132252978092,3,12,27,55.906667,56.274167,55.285556
2,2023-12-14 19:45:45+00:00,1714753328638672,4,25,35,71.425000,70.763200,70.128857
3,2023-12-20 06:50:23+00:00,2325282382443339,5,26,48,12.208000,9.889231,9.993958
4,2023-12-08 10:01:13+00:00,2880529151312391,2,6,6,60.320000,56.346667,56.346667
5,2023-12-18 16:22:20+00:00,3098369735660044,5,22,42,82.668000,80.505000,80.230000
6,2023-12-14 13:48:24+00:00,5030876361186394,2,6,12,105.965000,101.928333,95.103333
7,2023-12-10 03:20:19+00:00,5748320542153481,3,20,20,63.716667,67.382000,67.382000
8,2023-12-08 22:18:29+00:00,6544671314653569,1,7,7,36.640000,37.391429,37.391429
9,2023-12-05 12:09:44+00:00,6766605839627993,22,22,22,84.162727,84.162727,84.162727


##### Terminal feature table

Terminal table SQL query string:

In [12]:
create_terminal_batch_features_query = f"""
CREATE OR REPLACE TABLE `{TERMINALS_BQ_TABLE_URI}` AS
WITH
  -- query to join labels with features -------------------------------------------------------------------------------------------
  get_raw_table AS (
  SELECT
    raw_tx.TX_TS,
    raw_tx.TX_ID,
    raw_tx.CUSTOMER_ID,
    raw_tx.TERMINAL_ID,
    raw_tx.TX_AMOUNT,
    raw_lb.TX_FRAUD
  FROM (
    SELECT
      *
    FROM
      `{RAW_BQ_TRANSACTION_TABLE_URI}`
    WHERE
      DATE(TX_TS) BETWEEN DATE_SUB("{DATAPROCESSING_END_DATE}", INTERVAL 15 DAY) AND "{DATAPROCESSING_END_DATE}"
    ) raw_tx
  LEFT JOIN 
    `{RAW_BQ_LABELS_TABLE_URI}` as raw_lb
  ON raw_tx.TX_ID = raw_lb.TX_ID),

  # query to calculate TERMINAL spending behaviour --------------------------------------------------------------------------------
  get_variables_delay_window AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    
    # calc total amount of fraudulent tx and the total number of tx over the delay period per terminal (7 days - delay, expressed in seconds)
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_DELAY,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS NB_TX_DELAY,
      
    # calc total amount of fraudulent tx and the total number of tx over the delayed window per terminal (window + 7 days - delay, expressed in seconds)
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 691200 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_1_DELAY_WINDOW,
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_7_DELAY_WINDOW,
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1814400 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_14_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 691200 PRECEDING
      AND CURRENT ROW ) AS NB_TX_1_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS NB_TX_7_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1814400 PRECEDING
      AND CURRENT ROW ) AS NB_TX_14_DELAY_WINDOW,
  FROM get_raw_table),

  # query to calculate TERMINAL risk factors ---------------------------------------------------------------------------------------
  get_risk_factors AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    # calculate numerator of risk index
    NB_FRAUD_1_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_1DAY_WINDOW,
    NB_FRAUD_7_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_7DAY_WINDOW,
    NB_FRAUD_14_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_14DAY_WINDOW,
    # calculate denominator of risk index
    NB_TX_1_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_1DAY_WINDOW,
    NB_TX_7_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_7DAY_WINDOW,
    NB_TX_14_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_14DAY_WINDOW,
      FROM
    get_variables_delay_window),

  # query to calculate the TERMINAL risk index -------------------------------------------------------------------------------------
  get_risk_index AS (
    SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    TERMINAL_ID_NB_TX_1DAY_WINDOW,
    TERMINAL_ID_NB_TX_7DAY_WINDOW,
    TERMINAL_ID_NB_TX_14DAY_WINDOW,
    # calculate the risk index
    (TERMINAL_ID_NB_FRAUD_1DAY_WINDOW/(TERMINAL_ID_NB_TX_1DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_1DAY_WINDOW,
    (TERMINAL_ID_NB_FRAUD_7DAY_WINDOW/(TERMINAL_ID_NB_TX_7DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_7DAY_WINDOW,
    (TERMINAL_ID_NB_FRAUD_14DAY_WINDOW/(TERMINAL_ID_NB_TX_14DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_14DAY_WINDOW
    FROM get_risk_factors 
  )

# Create the table with CUSTOMER and TERMINAL features ----------------------------------------------------------------------------
SELECT
  PARSE_TIMESTAMP("%Y-%m-%d %H:%M:%S", FORMAT_TIMESTAMP("%Y-%m-%d %H:%M:%S", TX_TS, "UTC")) AS feature_ts,
  TERMINAL_ID AS terminal_id,
  CAST(TERMINAL_ID_NB_TX_1DAY_WINDOW AS INT64) AS terminal_id_nb_tx_1day_window,
  CAST(TERMINAL_ID_NB_TX_7DAY_WINDOW AS INT64) AS terminal_id_nb_tx_7day_window,
  CAST(TERMINAL_ID_NB_TX_14DAY_WINDOW AS INT64) AS terminal_id_nb_tx_14day_window,
  CAST(TERMINAL_ID_RISK_1DAY_WINDOW AS FLOAT64) AS terminal_id_risk_1day_window,
  CAST(TERMINAL_ID_RISK_7DAY_WINDOW AS FLOAT64) AS terminal_id_risk_7day_window,
  CAST(TERMINAL_ID_RISK_14DAY_WINDOW AS FLOAT64) AS terminal_id_risk_14day_window,
FROM
  get_risk_index
"""

##### Run the query 

You create the customer features table

In [13]:
run_bq_query(create_terminal_batch_features_query)

Finished job_id: c5c6a47e-2cfd-43ea-a12e-994cf24edaea


##### Inspect the result 

You can query some data rows to validate the result of the query

In [14]:
run_bq_query(
    f"SELECT * FROM `{TERMINALS_BQ_TABLE_URI}` LIMIT 10",
    show=True
)

Finished job_id: 9d629789-6156-473e-b914-6b478b722288


,feature_ts,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_7day_window,terminal_id_nb_tx_14day_window,terminal_id_risk_1day_window,terminal_id_risk_7day_window,terminal_id_risk_14day_window
0,2023-12-15 15:49:41+00:00,01596843,9,59,59,0.0,0.000000,0.000000
1,2023-12-18 13:23:23+00:00,16631216,27,155,155,0.0,0.006452,0.006452
2,2023-12-20 03:27:00+00:00,16631216,24,158,187,0.0,0.006329,0.005348
3,2023-12-18 08:39:08+00:00,18597292,24,128,128,0.0,0.046875,0.046875
4,2023-12-10 01:32:08+00:00,39619330,0,0,0,0.0,0.000000,0.000000
5,2023-12-09 12:55:20+00:00,42211721,0,0,0,0.0,0.000000,0.000000
6,2023-12-09 20:37:47+00:00,54126598,0,0,0,0.0,0.000000,0.000000
7,2023-12-13 16:51:30+00:00,66356805,19,29,29,0.0,0.000000,0.000000
8,2023-12-16 14:35:47+00:00,67320862,13,87,87,0.0,0.022988,0.022988
9,2023-12-17 03:04:24+00:00,68428753,20,106,106,0.1,0.028302,0.028302


#### Inspect BigQuery features tables

In [15]:
run_bq_query(
    f"SELECT * FROM `{CUSTOMERS_BQ_TABLE_URI}` LIMIT 5", show=True
)

Finished job_id: 7eac15ad-dde2-42a6-9bf9-6cbef0d2b1cf


,feature_ts,customer_id,customer_id_nb_tx_1day_window,customer_id_nb_tx_7day_window,customer_id_nb_tx_14day_window,customer_id_avg_amount_1day_window,customer_id_avg_amount_7day_window,customer_id_avg_amount_14day_window
0,2023-12-12 07:43:49+00:00,0250623170428445,5,23,23,82.226,85.656957,85.656957
1,2023-12-06 03:23:38+00:00,0896910678674863,2,2,2,94.665,94.665000,94.665000
2,2023-12-06 21:23:01+00:00,1211202768965460,2,5,5,11.285,15.336000,15.336000
3,2023-12-16 22:33:34+00:00,2325282382443339,5,22,43,9.102,9.715000,10.395814
4,2023-12-06 12:17:11+00:00,2421869738279148,2,3,3,61.720,65.166667,65.166667


In [16]:
run_bq_query(
    f"SELECT * FROM `{TERMINALS_BQ_TABLE_URI}` LIMIT 5", show=True
)

Finished job_id: 02cd5a4f-b0d4-45b5-902c-816bc8331f9c


,feature_ts,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_7day_window,terminal_id_nb_tx_14day_window,terminal_id_risk_1day_window,terminal_id_risk_7day_window,terminal_id_risk_14day_window
0,2023-12-15 15:49:41+00:00,01596843,9,59,59,0.0,0.000000,0.000000
1,2023-12-18 13:23:23+00:00,16631216,27,155,155,0.0,0.006452,0.006452
2,2023-12-20 03:27:00+00:00,16631216,24,158,187,0.0,0.006329,0.005348
3,2023-12-18 08:39:08+00:00,18597292,24,128,128,0.0,0.046875,0.046875
4,2023-12-10 01:32:08+00:00,39619330,0,0,0,0.0,0.000000,0.000000


Let's look at the final schema of the features table:

In [17]:
run_bq_query(f"""
SELECT column_name, data_type
FROM tx.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{CUSTOMERS_TABLE_NAME}'
""", show=True)

Finished job_id: e8cbbd06-a0c2-4269-935f-d03dc94ee18c


,column_name,data_type
0,feature_ts,TIMESTAMP
1,customer_id,STRING
2,customer_id_nb_tx_1day_window,INT64
3,customer_id_nb_tx_7day_window,INT64
4,customer_id_nb_tx_14day_window,INT64
5,customer_id_avg_amount_1day_window,FLOAT64
6,customer_id_avg_amount_7day_window,FLOAT64
7,customer_id_avg_amount_14day_window,FLOAT64


In [38]:
run_bq_query(f"""
SELECT column_name, data_type
FROM tx.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{TERMINALS_TABLE_NAME}'
""", show=True)

Finished job_id: 87a88172-6515-4123-b07b-6819475b4f01


,column_name,data_type
0,feature_ts,TIMESTAMP
1,terminal_id,STRING
2,terminal_id_nb_tx_1day_window,INT64
3,terminal_id_nb_tx_7day_window,INT64
4,terminal_id_nb_tx_14day_window,INT64
5,terminal_id_risk_1day_window,FLOAT64
6,terminal_id_risk_7day_window,FLOAT64
7,terminal_id_risk_14day_window,FLOAT64


In addition to feature engineering with SQL or with pandas (e.g BigQuery Dataframes), BQML also gives us the possiblity to perform in database transformations using [BQML Feature Engineering](https://cloud.google.com/bigquery/docs/manual-preprocessing)